In [1]:
from langchain.schema import Document
import json

# 파일 경로 
file_paths = {
    "term": "./metadata/term.json",
    "load_traffic_law": "./metadata/load_traffic_law.json",
    "modifier": "./metadata/modiflier.json",
    "car_case": "./metadata/car_to_car.json",
    "precedent": "./metadata/law.json"
}

# 교통사고 케이스용 필드 상수
CASE_ID = "사건 ID"
CASE_TITLE = "사건 제목"
CASE_SITUATION = "사고상황"
BASE_RATIO = "기본 과실비율"
MODIFIERS = "케이스별 과실비율 조정예시"
LAW_REFERENCES = "관련 법규"
PRECEDENT = "참고 판례"
REASON = "기본 과실비율 해설"

# JSON 로드 함수
def load_json(path):
    with open(path, 'r', encoding='utf-8') as f:
        return json.load(f)

# 리스트형 JSON 변환 (term, modifier, law_meta)
def convert_list_to_documents(data_list, doc_type):
    return [
        Document(page_content=json.dumps(item, ensure_ascii=False), metadata={"type": doc_type})
        for item in data_list
    ]

def convert_precedent_documents(data_list):
    return [
        Document(
            page_content=f"{item['court']} {item['case_id']} : {item['content']}",
            metadata={
                "court": item["court"],
                "case_id": item["case_id"],
            }
        ) for item in data_list
    ]

def convert_car_case_documents(data_list):
    documents = []

    def safe_value(value):
        if isinstance(value, list):
            return ", ".join(map(str, value))
        elif isinstance(value, dict):
            return json.dumps(value, ensure_ascii=False)
        elif value is None:
            return ""  # null도 허용 안 되므로 빈 문자열로 처리
        else:
            return str(value)

    for item in data_list:
        if not isinstance(item, dict):
            continue

        # page_content는 원본 전체 JSON 문자열
        content = json.dumps(item, ensure_ascii=False)

        # 기본 과실비율 해설이 리스트일 수 있음 → 문자열로 병합
        reason = item.get(REASON)
        if isinstance(reason, list):
            reason = "\n".join(map(str, reason))

        metadata = {
            "type": "car_case",
            "id": safe_value(item.get(CASE_ID)),
            "title": safe_value(item.get(CASE_TITLE)),
            "situation": safe_value(item.get(CASE_SITUATION)),
            "base_ratio": safe_value(item.get(BASE_RATIO)),
            "modifiers": safe_value(item.get(MODIFIERS)),
            "load_traffic_law": safe_value(item.get(LAW_REFERENCES)),
            "precedent": safe_value(item.get(PRECEDENT)),
            "reason": safe_value(reason)
        }

        documents.append(Document(page_content=content, metadata=metadata))
    return documents

# 도로교통법 law JSON → 문서화
def convert_law_json_to_documents(data_dict):
    documents = []

    def normalize(item):
        return json.dumps(item, ensure_ascii=False) if isinstance(item, dict) else str(item)

    for law_name, content in data_dict.items():
        if isinstance(content, dict):
            for clause, text in content.items():
                lines = [normalize(x) for x in (text if isinstance(text, list) else [text])]
                full_text = f"{law_name} {clause}\n" + "\n".join(lines)
                documents.append(Document(page_content=full_text, metadata={"type": "load_traffic_law"}))
        else:
            lines = [normalize(x) for x in (content if isinstance(content, list) else [content])]
            full_text = f"{law_name}\n" + "\n".join(lines)
            documents.append(Document(page_content=full_text, metadata={"type": "load_traffic_law"}))
    
    return documents


# 문서화 실행
term_docs = convert_list_to_documents(load_json(file_paths["term"]), "term")
modifier_docs = convert_list_to_documents(load_json(file_paths["modifier"]), "modifier")
precedent_docs = convert_precedent_documents(load_json(file_paths["precedent"]))
car_case_docs = convert_car_case_documents(load_json(file_paths["car_case"]))
load_traffic_law_docs = convert_law_json_to_documents(load_json(file_paths["load_traffic_law"]))


# 전체 문서 리스트
all_docs = term_docs + modifier_docs + car_case_docs + precedent_docs + load_traffic_law_docs
all_docs = precedent_docs

In [2]:
from langchain.vectorstores import Chroma  # persist 지원
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 1. 청크 크기 조정 (500~1000 권장)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)

# 2. 문서 분할
all_splits = text_splitter.split_documents(all_docs)

embedding_model = OpenAIEmbeddings()

# 4. Chroma DB에 배치 처리로 저장
batch_size = 100  # 한 번에 처리할 청크 수
vectorstore = Chroma.from_documents(
    documents=all_splits[:batch_size],  # 첫 배치
    embedding=embedding_model,
    persist_directory="./vectordb"
)

# 남은 청크를 순차적으로 추가
for i in range(batch_size, len(all_splits), batch_size):
    batch = all_splits[i:i+batch_size]
    vectorstore.add_documents(
        documents=batch,
        embedding=embedding_model
    )

vectorstore.persist()  


C:\Users\Playdata\AppData\Local\Temp\ipykernel_4728\3046172222.py:34: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [3]:
vectorstore.get()

{'ids': ['3fc7a2bb-f661-4d09-bc43-776e929ebeaa',
  'd3b6475a-a4c4-419e-9f49-149f888d878e',
  'dedab9ed-918c-473a-ba04-43ec1d7f3145',
  'aa3e7ced-5e86-4b8d-abc4-4641a662cc8a',
  '3df64a02-5841-4143-8c7d-5380e6a8ff96',
  'b60f9a39-7beb-43ad-8b60-bdc2a2c8881d',
  '3a8f3be2-0996-4b80-b4e9-45783c417182',
  'eba21538-c68c-40c6-83dd-65097af15843',
  '1e94c18f-b3f7-478f-ab93-3bed80b78edc',
  '48ee1974-22d8-4e2f-8799-9a2131028075',
  'a0370b42-9399-444b-800b-96f6a855f355',
  '808cdb8b-dedd-4e5a-8a95-4b908358c77d',
  '93f1365c-51ce-43e3-a891-dec8e2d5e7c1',
  '31c71bcd-5599-4b2e-a60b-723d79d92243',
  'c5dafb2b-7c13-49a9-a2f5-e04fec60dc64',
  '07d1ea6f-4834-4b78-abcb-ed5015cbc36d',
  '26f31fd5-356f-4422-bb8d-887f1d6037b7',
  '473ab361-efeb-4b82-a33e-3415f0e2400a',
  '2f5fae7f-6337-476c-ad91-63fdb7ad2789',
  'df0bdcaf-e8f2-4f7a-b086-fa5f59407f73',
  '997e8fb1-ea12-48bf-afe8-8cee6fb09947',
  'eb539d73-78e1-4bfa-891a-395b35eb968f',
  '2d34a2ee-2990-4ef0-8ddf-b903bdffe766',
  'b2c5043e-78ee-48df-98cf-

In [4]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain.vectorstores import Chroma

# 1. LLM 설정
model = ChatOpenAI(model='gpt-4o', temperature=0.5)

# 2. 프롬프트 템플릿
prompt = PromptTemplate(
    template="""
다음 '사고 상황 설명'에 대해 '문서'의 내용만 참고하여 과실 비율 및 법적 판단을 생성해 주세요.

사고 상황: {question}

문서 내용: {context}

답변 형식:
1. 과실비율: A차량 xx% vs B차량 xx%
2. 사고 원인 및 판단 근거:
   - [사고의 전개, 각 차량의 행위, 과실 요소 등을 구체적으로 설명]
3. 관련 법률 조항:
   - [예: 도로교통법 제10조 제2항, 제27조 제1항 등]
4. 참고 판례:
   - [예: 대법원 2023다12345, 서울중앙지법 2022가단123456 등]

조건:
- 반드시 문서 내용만 참고해 판단하세요.
- 법률 조항과 판례가 명시되어 있지 않으면 유사하거나 추정 가능한 근거를 제시해도 됩니다.
- 전체 답변은 포맷에 맞게 간결하고 전문적으로 작성하세요.
""",
    input_variables=["question", "context"]
)

# 3. 리트리버 설정
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# 4. QA 체인 재구성 (수정 부분)
qa_chain = RetrievalQA.from_chain_type(
    llm=model,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)

# 5. 실제 질의 실행
query = "B차량이 비보호좌회전 표지가 없는 교차로에서 녹색등(좌회전 화살표 없음)에 좌회전 시도, 직진하는 A차량과 충돌한 사고가 발생했어. 과실비율이 어떻게 돼??"

res = qa_chain.invoke({"query": query})

# 6. 출력
print("✅ 답변:\n", res["result"])


✅ 답변:
 1. 과실비율: A차량 20% vs B차량 80%

2. 사고 원인 및 판단 근거:
   - 사고는 B차량이 비보호좌회전 표지가 없는 교차로에서 녹색등에 좌회전을 시도하면서 직진하는 A차량과 충돌하여 발생했습니다. 비보호좌회전 상황에서는 좌회전 차량이 직진 차량의 통행을 방해하지 않도록 주의해야 할 의무가 있습니다. 문서 내용에 따르면, 비보호좌회전 차량의 과실이 높은 경우가 많으며, 특히 직진 차량이 규정 속도를 준수하고 정상적으로 진행 중인 경우, 좌회전 차량의 과실이 더 크게 인정됩니다. 따라서 B차량의 과실이 80%로 판단됩니다.

3. 관련 법률 조항:
   - 도로교통법 제27조 제1항: 모든 차의 운전자는 교차로에서 좌회전할 때 반대 방향에서 직진하는 차량의 통행을 방해하지 않도록 해야 한다.

4. 참고 판례:
   - 서울중앙지방법원 2017나65373: 비보호좌회전 차량의 과실이 높게 인정된 사례.
   - 서울지방법원북부지원 92가합10474: 좌회전 차량의 주의 의무를 게을리한 경우 높은 과실이 인정된 사례.


In [5]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

detail_prompt = PromptTemplate(
    template="""
아래 문서 내용을 바탕으로 사용자가 물어본 용어나 법률 조항, 판례에 대해 정확하고 간결하게 설명해 주세요.

질문: {question}

문서: {context}

답변 형식:
- 용어/조항 정의: [정확한 설명]
- 출처가 명시된 경우: 관련 법률/조문 번호/판례명을 반드시 포함

답변:
""",
    input_variables=["question", "context"]
)

detail_chain = RetrievalQA.from_chain_type(
    llm=model,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": detail_prompt}
)

# run은 question 키를 기준으로 실행됨!
res2 = detail_chain.run("보행자우선도로가 뭔지 설명해줘")

print("✅ 기능② 답변:\n", res2)


C:\Users\Playdata\AppData\Local\Temp\ipykernel_4728\1518626063.py:29: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  res2 = detail_chain.run("보행자우선도로가 뭔지 설명해줘")


✅ 기능② 답변:
 - 용어/조항 정의: 보행자우선도로는 보행자의 안전과 편의를 최우선으로 고려하여 설계된 도로를 의미합니다. 이 도로에서는 보행자가 차량보다 우선권을 가지며, 차량은 보행자의 안전을 보장하기 위해 저속으로 운행해야 하고 보행자에게 충분한 주의를 기울여야 합니다. 보행자우선도로는 주로 주거 지역이나 학교 주변 등 보행자의 통행이 많은 지역에 설치됩니다.

- 출처가 명시된 경우: 관련 법률/조문 번호/판례명이 문서에 명시되어 있지 않으므로 생략합니다.


In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
import re

# LLM 설정
model = ChatOpenAI(model='gpt-4o', temperature=0.5)


# 기능 1번. 사고 상황 판단 프롬프트
prompt = PromptTemplate(
    template="""
너는 교통사고 과실 판단 전문가야.
아래 '사고 상황'을 분석하여 핵심 요소를 구조화하고, 반드시 문서 내에서 가장 유사한 사례(case)를 찾아 과실비율을 판단해줘.

---
사고 상황 원문:
{question}

➤ 사고 상황 요약 (다음 항목 기준):
- A차량 신호 및 진행 방식:
- B차량 신호 및 진행 방식:
- 충돌 방식 및 위치:
- 교차로/신호기 유무 등 도로 환경:

문서:
{context}

출력 형식:
1. 과실비율: A차량 xx% vs B차량 xx%
2. 판단 근거 요약
3. 적용 법률 (다음 형식으로 기재)
   - [법률명] 제[조]조 [항] (예: 도로교통법 제5조 제1항)
   - 여러 개면 각각 별도 줄로 나열
4. 참고 판례 (다음 형식으로 기재)
   - [법원명] [사건번호] (예: 대법원 2002다38767)
   - 여러 개면 각각 별도 줄로 나열

조건:
- 반드시 문서 내 유사 사례를 기반으로 판단해야 해.
- 사고 상황과 가장 유사한 사례를 먼저 정확히 찾고, 그 사례를 중심으로 판단 근거를 제시해.
- 직접적인 사례가 없으면, 가장 유사한 사례를 근거로 판단하고 그 차이를 설명해.
- 과실비율은 명확한 수치로 제시해.
- 법령, 판례는 반드시 문서에서 인용된 것을 정확한 형식으로 명시해.
- 무조건적인 추론이나 일반적인 상식은 사용하지 말고, 문서에 있는 정보로만 판단해.
- 판례명을 포함한 질문인 경우, 문서에서 사건번호와 법원명이 일치하는 문장을 찾아 정확히 인용해줘.
""",
    input_variables=["question", "context"]
)


# 기능 2번. 용어/법률 설명 프롬프트
# detail_prompt = PromptTemplate(
#     template="""
# 너는 교통사고와 법률 해설 전문 챗봇이야.

# 아래 문서 내용을 바탕으로 사용자 질문에 정확하고 간결하게 답변해줘.

# 질문:
# {question}

# --- 문서 내용 ---
# {context}
# ------------------

# 📌 답변 작성 규칙:
# 1. 질문에 판례명(법원명 + 사건번호)이 포함된 경우:
#    - 반드시 위 문서 내에서 **해당 사건번호가 포함된 문장**을 직접 인용해 설명해.
#    - 문서에 해당 판례가 없으면 "**해당 판례는 문서에 포함되어 있지 않습니다**"라고 말해.

# 2. 질문이 용어나 법률 조항에 대한 경우:
#    - 반드시 위 문서에 포함된 문장을 바탕으로 정의하거나 설명해.
#    - 법률명, 조문 번호, 관련 판례명을 반드시 함께 명시해.

# 3. 문서에 없는 정보는 절대로 추측하거나 생성하지 마.

# ✍️ 출력 형식:
# - 정의 또는 설명: [내용]
# - 출처: [법률/조문 번호/판례명]

# 답변:
# """,
#     input_variables=["question", "context"]
# )

detail_prompt = PromptTemplate(
    template="""
너는 교통사고와 법률 해설 전문 챗봇이야.

아래 문서 내용은 교통사고 관련 판례, 법률 조항, 용어 정의가 포함된 데이터야.
문서는 다음과 같은 형식의 메타데이터를 포함하고 있어:

📘 판례 메타데이터 예시:
- case_id: '2006나4320'
- court: '서울고등법원'
- type: 'judgment'

📘 법률 조항 예시:
- 도로교통법 제5조 제1항
- 도로교통법 제25조 제5항

📘 용어 정의는 '과실', '중대한 과실', '현저한 과실' 등 문장으로 설명됨

사용자의 질문을 보고 다음 중 하나에 해당하도록 답변해줘:
1. 판례(case_id)가 질문에 포함되면, 문서에서 해당 case_id가 포함된 문장을 찾아서 **직접 인용**하여 설명해.
2. 법률 조항(예: 도로교통법 제x조 제y항)이 포함된 경우, 반드시 **문서에 있는 원문**을 바탕으로 설명하고 법률명을 정확히 명시해.
3. 용어나 개념 정의 질문일 경우, 문서에서 해당 용어가 포함된 문장을 찾아 그 문장만 바탕으로 설명해.

⚠️ 문서에 포함되지 않은 정보는 생성하지 마.
⚠️ 질문과 관련된 메타데이터가 없으면, “문서에 포함되어 있지 않습니다”라고 명시해.

✍️ 출력 형식:
- 정의 또는 설명: [내용]
- 출처: [법률/조문 번호/판례명]

질문:
{question}

--- 문서 내용 ---
{context}
------------------

답변:
""",
    input_variables=["question", "context"]
)


# 리트리버 설정
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

# 사고 상황 판단 체인
accident_chain = RetrievalQA.from_chain_type(
    llm=model,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt}
)

# 설명용 체인
detail_chain = RetrievalQA.from_chain_type(
    llm=model,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": detail_prompt}
)

# 통합 실행 함수
# 🔍 판례명 추출
def extract_case_id(text: str):
    match = re.search(r"(대법원|[\w]+고등법원|[\w]+지방법원)\s?\d{4}[가-힣]?\d+", text)
    return match.group() if match else None

# 🔍 법원명 추출
def extract_court_name(text: str):
    match = re.search(r"(대법원|[\w]+고등법원|[\w]+지방법원)", text)
    return match.group() if match else None

# 🧠 사고상황 vs 용어 설명 구분
def classify_query(user_input: str) -> str:
    accident_keywords = ["충돌", "사고", "과실", "A차량", "B차량", "신호위반", "부딪힘", "진입", "교차로"]
    explanation_keywords = ["무엇", "뭐야", "정의", "설명", "판례", "조항", "법원", "법", "도로교통법", "무슨 뜻", "의미", "용어", "이란"]

    if any(word in user_input for word in explanation_keywords):
        return "detail"
    if any(word in user_input for word in accident_keywords):
        return "accident"
    return "detail"

# ✨ context 최적화 함수
def build_context_from_matched_docs(matched_docs, key_phrase=None, limit=3):
    if key_phrase:
        primary_docs = [doc for doc in matched_docs if key_phrase in doc.page_content]
        other_docs = [doc for doc in matched_docs if key_phrase not in doc.page_content]
        ordered_docs = primary_docs + other_docs[:limit - len(primary_docs)]
    else:
        ordered_docs = matched_docs[:limit]

    # ✅ 각 문서 최대 800자 제한
    context = "\n\n".join(doc.page_content[:800] for doc in ordered_docs)
    return context

# ✅ 메인 실행 함수
def run_query(user_input: str):
    case_id = extract_case_id(user_input)
    court = extract_court_name(user_input)

    # ① 판례명 포함 질문
    if case_id:
        matched_docs = [doc for doc in all_docs if case_id in doc.page_content]

        if matched_docs:
            context = build_context_from_matched_docs(matched_docs, key_phrase=case_id)
            prompt_text = detail_prompt.format(question=user_input, context=context)
            result = model.invoke(prompt_text)
            print("📘 판례 직접 검색 결과:\n", result.content.strip())
            return
        else:
            print(f"❌ 해당 판례 '{case_id}'는 문서에 포함되어 있지 않습니다.")
            return

    # ② 법원명 기반 질문
    if court:
        matched_docs = [doc for doc in all_docs if doc.metadata.get("court") == court]

        if matched_docs:
            context = build_context_from_matched_docs(matched_docs, key_phrase=None)
            prompt_text = detail_prompt.format(question=user_input, context=context)
            result = model.invoke(prompt_text)
            print(f"📘 '{court}' 소속 판례 검색 결과:\n", result.content.strip())
            return
        else:
            print(f"❌ '{court}' 관련 문서가 포함되어 있지 않습니다.")
            return

    # ③ 일반 질문: 사고 상황 판단 or 용어 설명
    category = classify_query(user_input)

    if category == "accident":
        result = accident_chain.run(user_input)
        print("✅ 사고 상황 판단 결과:\n", result)
    else:
        result = detail_chain.run(user_input)
        print("📘 용어/법률 설명 결과:\n", result)

### 테스트

In [7]:
run_query("A차량이 적색신호에 직진 중이고, B차량은 좌회전 신호 없이 녹색등에서 좌회전 중 충돌한 사고야. 과실비율은?")

✅ 사고 상황 판단 결과:
 사고 상황을 분석하고 핵심 요소를 구조화한 결과는 다음과 같습니다:

➤ 사고 상황 요약:
- A차량 신호 및 진행 방식: 적색신호에 직진
- B차량 신호 및 진행 방식: 좌회전 신호 없이 녹색등에서 좌회전
- 충돌 방식 및 위치: 교차로에서 충돌
- 교차로/신호기 유무 등 도로 환경: 신호등 있는 교차로

문서 내 가장 유사한 사례는 "인천지방법원 97가단8474" 사건입니다. 이 사건은 신호기 있는 교차로에서 B차량이 적색신호에 좌회전을 하고, 반대방향에서 적색신호에 직진하던 A이륜차와 충돌한 사고입니다. 이 사례에서 B차량의 과실이 65%로 판단되었습니다.

1. 과실비율: A차량 35% vs B차량 65%

2. 판단 근거 요약:
   - A차량은 적색신호에 직진하여 신호 위반을 하였고,
   - B차량은 좌회전 신호 없이 녹색등에서 좌회전을 하여 신호 위반을 하였습니다.
   - 유사한 사례인 인천지방법원 97가단8474에서 B차량의 과실이 65%로 인정된 점을 고려하여, 본 사고에서도 B차량의 과실이 더 크다고 판단됩니다.

3. 적용 법률:
   - 도로교통법 제5조 제1항 (신호에 따를 의무)
   - 도로교통법 제6조 제1항 (신호위반의 금지)

4. 참고 판례:
   - 인천지방법원 97가단8474


In [8]:
run_query("비보호 좌회전이 뭔지 설명해줘")

📘 용어/법률 설명 결과:
 - 정의 또는 설명: 비보호 좌회전이란, 비보호 좌회전 구역에서 좌회전하는 차량은 반대방향에서 차량 진행신호에 따라 직진해 오는 차량에 주의하면서 직진 차량의 통행에 방해되지 않는 방법으로 좌회전을 조심스럽게 하여야 한다. 반대방향에서 직진하는 차량에게 방해가 되지 않도록 유의하여 좌회전할 의무가 있다.
- 출처: [서울중앙지방법원 2017나65373, 대구지방법원 2016나309440]


In [9]:
run_query("대법원 92도2077 판결문 내용을 알려줘")

📘 '대법원' 소속 판례 검색 결과:
 - 정의 또는 설명: 차량의 운전자로서는 횡단보도의 신호가 적색인 상태에서 반대 차로 상에 정지하여 있는 차량의 뒤로 보행자가 건너오지 않을 것이라고 신뢰하는 것이 당연하고 그렇지 아니할 사태까지 예상하여 그에 대한 주의의무를 다하여야 한다고는 할 수 없다.
- 출처: 대법원 92도2077


In [10]:
run_query("서울남부지방법원에는 어떤 판례가 있어?")

📘 '서울남부지방법원' 소속 판례 검색 결과:
 - 정의 또는 설명: [서울남부지방법원에는 다음과 같은 판례가 있습니다: 
  1. 2018나64069: 신호등이 있는 ‘ㅏ’자형 교차로에서의 사고로, B이륜차 운전자가 사망한 사건이며, B의 과실이 55%로 판정되었습니다.
  2. 2015가단227332: 주간에 신호등 및 비보호 좌회전 표지가 있는 삼거리에서 발생한 사고로, B차량의 과실이 10%로 판정되었습니다.
  3. 2018나11137: B 직진차량과 A 비보호 좌회전 차량 간의 사고로, B차량의 과실이 0%로 판정되었습니다.]
- 출처: [서울남부지방법원 판례]


In [11]:
run_query("A차가 직진 중이고 B차가 좌회전하다 충돌했을 때 과실은?")

✅ 사고 상황 판단 결과:
 1. 과실비율: A차량 0% vs B차량 100%

2. 판단 근거 요약:
   - A차량은 직진 중이었고, B차량은 좌회전 중이었습니다.
   - 문서에서 가장 유사한 사례는 "서울중앙지방법원 2016나76772"입니다. 이 사례에서는 A차량이 정상적으로 직진 중이었고, B차량이 좌회전 금지 차로에서 좌회전하다가 사고가 발생한 경우 A차량의 과실이 0%로 판단되었습니다.
   - 해당 사례와 유사하게, A차량이 직진 중이고 B차량이 좌회전 중인 상황에서 A차량의 과실이 없다고 판단할 수 있습니다.

3. 적용 법률:
   - 도로교통법 제5조 제1항
   - 도로교통법 제25조 제2항

4. 참고 판례:
   - 서울중앙지방법원 2016나76772


In [12]:
run_query("서울남부지방법원 2018나64069 판결문 내용을 알려줘")

📘 판례 직접 검색 결과:
 - 정의 또는 설명: 신호등이 있는 ‘ㅏ’자형 교차로에서 전방 신호기 황색등화에 교차로를 가속 진입한 A차량과 반대편 차선 좌회전 금지구역에서 적색 신호에 좌회전을 한 B이륜차 간의 충돌로 B이륜차 운전자가 사망한 사고: B과실 55%(안전모 미착용 포함)
- 출처: [서울남부지방법원 2018나64069]


In [15]:
import random
json_precedent = load_json(file_paths["precedent"])
random_precedent = random.sample(json_precedent, 10)

for precedent in random_precedent:
    run_query(f"\n{precedent['court']} {precedent['case_id']} 판결문 내용을 알려줘")
    print()

📘 판례 직접 검색 결과:
 - 정의 또는 설명: 원고 차량이 편도 4차로 도로의 4차로를 따라 서행하다가 길가장자리를 따라 우회전하는 순간 피고 이륜차가 원고 차량을 뒤따르던 차량과 인도 사이 틈으로 그 차량을 빠른 속도로 추월한 다음 원고 차량까지 추월하고자 원고 차량과 인도 사이 틈으로 직진하다가 원고 차량을 충돌한 사고로, 피고 이륜차의 앞지르기 방법 위반의 전적인 과실로 발생한 사고로 판단함. 원고차량 과실 0%, 피고 이륜차 과실 100%.
- 출처: 서울남부지방법원 2018나50626

📘 판례 직접 검색 결과:
 - 정의 또는 설명: 도로교통법 제20조의2는 “모든 차의 운전자는 다음 각 호의 1에 해당하는 곳에서는 다른 차를 앞지르지 못한다.”고 규정하여 일정한 장소에서의 앞지르기를 금지하고 있으므로, 같은 조의 각 호에 해당하는 곳에서는 도로교통법 제18조에 의하여 앞차가 진로를 양보하였다 하더라도 앞지르기를 할 수 없다.
- 출처: 대법원 2004도8062

📘 '서울민사지방법원' 소속 판례 검색 결과:
 - 정의 또는 설명: 야간에 편도2차로의 사거리(十자) 교차로에서 A차량(덤프트럭)이 왼쪽 교차도로에서 2차로를 따라 주행하던 중 정지선 5미터 앞에서 예비신호를 받고도 정차하거나 주변 차량들의 움직임을 잘 살피지 아니하고 그대로 직진하여 통과한 과실로, 오른쪽 교차도로에서 신호에 따라 안전하게 운전해야 할 주의의무에 위반하여 예비신호를 받고도 그대로 직진하던 B차량(버스)의 왼쪽 앞부분을 위 덤프트럭의 오른쪽 앞부분으로 충돌한 사고 : B과실 50%
- 출처: 서울민사지방법원 92가합79931

📘 '의정부지방법원' 소속 판례 검색 결과:
 - 정의 또는 설명: 편도2차로 도로에서 피고는 2차로 우측의 갓길로 진로를 변경하여 진행하였고, 원고는 피고 차량 앞쪽 갓길로 진로를 변경하던 중 사고가 발생함, 원고 60%: 피고 40%
- 출처: 의정부지방법원 2016가단126159

📘 판례 직접 검색 결과:
 - 정의 또는 설명: [피고 차